# Pair Programming ANOVA

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.


In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [3]:
df = pd.read_csv("data/df_norm_estand.csv", index_col = 0)
df.sample(5)

,age,sex,bmi,children,smoker,region,charges,charges_Sklearn,age_robust,bmi_robust,children_robust
1051,22,female,27.1,0,no,southwest,2154.361,0.030958,-0.68,-0.328554,-0.5
757,51,female,34.2,1,no,southwest,9872.701,0.262386,0.48,0.556733,0.0
566,51,male,39.7,1,no,southwest,9391.346,0.247953,0.48,1.242519,0.0
743,63,male,33.1,0,no,southwest,13393.756,0.367962,0.96,0.419576,-0.5
829,19,female,23.4,2,no,southwest,2913.569,0.053722,-0.80,-0.789900,0.5


### Empezamos con el método OLS

In [4]:
lm = ols('charges_Sklearn ~ age + sex + bmi + children + smoker + region',data = df).fit()

In [5]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
sex,1.0,0.030904,0.030904,1.645281,1.998524e-01
smoker,1.0,20.484783,20.484783,1090.577381,2.986538e-170
region,3.0,0.232534,0.077511,4.126584,6.360910e-03
age,1.0,12.904397,12.904397,687.009633,6.904498e-120
bmi,1.0,0.155038,0.155038,8.253995,4.138508e-03
children,1.0,0.272009,0.272009,14.481332,1.487623e-04
Residual,1189.0,22.333497,0.018783,NaN,NaN


SI miramos el p-valor, al ser todos menores que el 0.05 rechazamos la H0 (que dice que la variable no tiene efecto sobre la variable respuesta) y vemos que todas influyen en la variable respuesta.

## Método summary 

In [6]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     226.8
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          4.49e-233
Time:                        19:44:04   Log-Likelihood:                 685.52
No. Observations:                1198   AIC:                            -1353.
Df Residuals:                    1189   BIC:                            -1307.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.1199      0.024     -4.919      0.000      -0.168      -0.072
sex[T.male]            -0.0109      0.008     -1.374      0.170      -0.026       0.005
smoker[T.yes]           0.4391      0.013     34.116      0.000       0.414       0.464
region[T.northwest]    -0.0082      0.011     -0.737      0.461      -0.030       0.014
region[T.southeast]    -0.0315      0.011     -2.736      0.006      -0.054      -0.009
region[T.southwest]    -0.0393      0.011     -3.458      0.001      -0.062      -0.017
age                     0.0073      0.000     25.540      0.000       0.007       0.008
bmi                     0.0020      0.001      2.875      0.004       0.001       0.003
children                0.0124      0.003      3.805      0.000       0.006       0.019
==============================================================================
Omnibus:                      750.552   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5243.984
Skew:                           3.011   Prob(JB):                         0.00
Kurtosis:                      11.294   Cond. No.                         322.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

El r-squared es el porcentaje de variación de la variable respuesta. En nuestro caso tenemos un 0.60, lo que significa que nuestras variables predictoras explican el 60% de la variación de los precios de nuestro seguro médico.

El coef nos indica los cambios medios en la variable respuesta para una unidad de cambio en la variable predictora

El p.valor nos dice que nuestras variables influyen, son menores que 0.05 menos en sex y region. Asi que cogeríamos todas nuestras variables menos esas dos.

El std err es el error estándar del coeficiente y cuanto menor sea, más preciso. En nuestro caso son todos pequeños.


## Vamos a ver con efectos aditivos 
Omitimos las interpretaciones ya que solo lo hemos hecho para ver este método.

In [7]:
lm = ols('charges_Sklearn ~ age_robust * bmi_robust * children_robust * smoker * sex * region',data = df).fit()

In [8]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,1.0,20.460507,20.460507,1171.461422,5.173345e-174
sex,1.0,0.055180,0.055180,3.159309,7.577911e-02
region,3.0,0.232534,0.077511,4.437896,4.149743e-03
smoker:sex,1.0,0.008091,0.008091,0.463231,4.962651e-01
smoker:region,3.0,0.128678,0.042893,2.455809,6.166103e-02
...,...,...,...,...,...
age_robust:bmi_robust:children_robust:smoker:sex,1.0,0.011157,0.011157,0.638804,4.243216e-01
age_robust:bmi_robust:children_robust:smoker:region,3.0,0.013274,0.004425,0.253328,8.589923e-01
age_robust:bmi_robust:children_robust:sex:region,3.0,0.040479,0.013493,0.772537,5.094273e-01
age_robust:bmi_robust:children_robust:smoker:sex:region,3.0,0.089021,0.029674,1.698957,1.655262e-01


In [9]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     17.01
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          1.21e-182
Time:                        19:44:05   Log-Likelihood:                 792.25
No. Observations:                1198   AIC:                            -1329.
Df Residuals:                    1070   BIC:                            -677.2
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================================================
                                                                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                               0.2502      0.012     21.384      0.000       0.227       0.273
smoker[T.yes]                                                                           0.4057      0.119      3.400      0.001       0.172       0.640
sex[T.male]                                                                            -0.0263      0.017     -1.538      0.124      -0.060       0.007
region[T.northwest]                                                                    -0.0307      0.017     -1.857      0.064      -0.063       0.002
region[T.southeast]                                                                    -0.0307      0.017     -1.766      0.078      -0.065       0.003
region[T.southwest]                                                                    -0.0563      0.016     -3.417      0.001      -0.089      -0.024
smoker[T.yes]:sex[T.male]                                                               0.0568      0.132      0.431      0.666      -0.201       0.315
smoker[T.yes]:region[T.northwest]                                                      -0.0632      0.143     -0.440      0.660      -0.345       0.218
smoker[T.yes]:region[T.southeast]                                                       0.0704      0.134      0.524      0.600      -0.193       0.334
smoker[T.yes]:region[T.southwest]                                                       1.4556      0.486      2.997      0.003       0.503       2.408
sex[T.male]:region[T.northwest]                                                         0.0221      0.024      0.923      0.356      -0.025       0.069
sex[T.male]:region[T.southeast]                                                         0.0114      0.026      0.443      0.658      -0.039       0.062
sex[T.male]:region[T.southwest]                                                         0.0199      0.024      0.825      0.410      -0.027       0.067
smoker[T.yes]:sex[T.male]:region[T.northwest]                                           0.0095      0.194      0.049      0.961      -0.372       0.391
smoker[T.yes]:sex[T.male]:region[T.southeast]                                          -0.1475      0.180     -0.821      0.412      -0.500       0.205
smoker[T.yes]:sex[T.male]:region[T.southwest]                                          -1.4645      0.493     -2.973      0.003      -2.431      -0.498
age_robust                                                                              0.1526      0.022      6.923      0.000       0.109       0.196
age_robust:smoker[T.yes]                   

## Vemos con efectos no aditivos pero con el Robust

In [10]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'charges_Sklearn', 'age_robust', 'bmi_robust', 'children_robust'],
      dtype='object')

In [11]:
lm = ols('charges_Sklearn ~ age_robust + bmi_robust + children_robust + smoker + sex + region',data = df).fit()

In [12]:
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
smoker,1.0,20.460507,20.460507,1089.284975,4.185267e-170
sex,1.0,0.055180,0.055180,2.937688,8.679514e-02
region,3.0,0.232534,0.077511,4.126584,6.360910e-03
age_robust,1.0,12.904397,12.904397,687.009633,6.904498e-120
bmi_robust,1.0,0.155038,0.155038,8.253995,4.138508e-03
children_robust,1.0,0.272009,0.272009,14.481332,1.487623e-04
Residual,1189.0,22.333497,0.018783,NaN,NaN


Igual que antes, parece que todas nuestras variables tienen efecto osbre la respuesta, al ser menor que 0.05.

In [13]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        charges_Sklearn   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     226.8
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          4.49e-233
Time:                        19:44:06   Log-Likelihood:                 685.52
No. Observations:                1198   AIC:                            -1353.
Df Residuals:                    1189   BIC:                            -1307.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.2368      0.009     26.272      0.000       0.219       0.255
smoker[T.yes]           0.4391      0.013     34.116      0.000       0.414       0.464
sex[T.male]            -0.0109      0.008     -1.374      0.170      -0.026       0.005
region[T.northwest]    -0.0082      0.011     -0.737      0.461      -0.030       0.014
region[T.southeast]    -0.0315      0.011     -2.736      0.006      -0.054      -0.009
region[T.southwest]    -0.0393      0.011     -3.458      0.001      -0.062      -0.017
age_robust              0.1817      0.007     25.540      0.000       0.168       0.196
bmi_robust              0.0164      0.006      2.875      0.004       0.005       0.028
children_robust         0.0249      0.007      3.805      0.000       0.012       0.038
==============================================================================
Omnibus:                      750.552   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5243.984
Skew:                           3.011   Prob(JB):                         0.00
Kurtosis:                      11.294   Cond. No.                         5.49
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Intepretación

El r-squared es el porcentaje de la variacion de la variable respuesta. ES decir, nuestras variables predictoras explican el 60,4% de la variacion en los precios del seguro medico (charges). ES un valor bajo.

El coef nos indica los cambios medios en la variable respuesta para una unidad de cambio en la variable predictora. Age, bmi y children tienen un coeficiente positivo, a mayor edad, bmi o hijos, mayores primas. Todas tienen una relación postiva, menos las de 'region', que es negativa.

El p.valor nos dice que nuestras variables influyen, son menores que 0.05 menos en sex y region. Asi que cogeríamos todas nuestras variables menos esas dos.

El std err es el error estándar del coeficiente y cuanto menor sea, más preciso. En nuestro caso son todos bastante bajos.

### En  conclusión, vemos que las variables region y sex  no influyen en la variable respuesta. Llegado el momento de hacer la regresión, las eliminaremos del dataframe para ajustar mejor el modelo.

In [18]:
df2.to_csv('data/df_tras_anova.csv')